루브릭
1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.  
구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.  
2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.  
seq2seq 모델 훈련결과를 그래프로 출력해보고, validation loss그래프가 우하향하는 경향성을 보이며 학습이 진행됨이 확인되었다.  
3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.  
테스트용 디코더 모델이 정상적으로 만들어졌으며, input(영어)와 output(프랑스어) 모두 한글로 번역해서 결과를 출력해보았고, 둘의 내용이 유사함을 확인하였다.  

# 단어 Level 번역기 만들기
데이터는 상위 33,000개의 샘플만 사용  
1.정제, 정규화, 전처리  
구두점 분리, 소문자 전환, 띄어쓰기 단위 토큰화  
2.디코더 문장에 시작 및 종료 토큰 삽입  
3.케라스 토크나이저로 텍스트 숫자 변환  
4.임배딩 레이어 사용하기(노드 10-10 step 4 참조  
5.모델 구현하기  
sparse categorical entropy loss사용  
6.모델 평가하기  
  
### 데이터 불러오기 및 전처리

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
# lines.sample(5) #샘플 5개 출력
lines.head(5)

전체 샘플의 수 : 197463


,eng,fra,cc
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [3]:
# 3만 3천개의 데이터만 사용한다.(train 30000, test 3000)
lines = lines[['eng', 'fra']][:33000]
lines.head()

,eng,fra
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [4]:
# 시작 토큰과 종료 토큰 추가
sos_token = '<sos>'
eos_token = '<eos>'
# sos_token과 eos_token사이에 ' '를 추가하여야 토그나이저에서 문제를 일으키지 않는다.
lines.fra = lines.fra.apply(lambda x : sos_token + ' ' + x + ' '+ eos_token)
print('전체 샘플의 수 :',len(lines))
lines.head(10)

전체 샘플의 수 : 33000


,eng,fra
0,Go.,<sos> Va ! <eos>
1,Go.,<sos> Marche. <eos>
2,Go.,<sos> En route ! <eos>
3,Go.,<sos> Bouge ! <eos>
4,Hi.,<sos> Salut ! <eos>
5,Hi.,<sos> Salut. <eos>
6,Run!,<sos> Cours ! <eos>
7,Run!,<sos> Courez ! <eos>
8,Run!,<sos> Prenez vos jambes à vos cous ! <eos>
9,Run!,<sos> File ! <eos>


In [5]:
# 모든 대문자 소문자 변환
lines['eng']=lines['eng'].str.lower()
lines['fra']=lines['fra'].str.lower()
lines.head(10)

,eng,fra
0,go.,<sos> va ! <eos>
1,go.,<sos> marche. <eos>
2,go.,<sos> en route ! <eos>
3,go.,<sos> bouge ! <eos>
4,hi.,<sos> salut ! <eos>
5,hi.,<sos> salut. <eos>
6,run!,<sos> cours ! <eos>
7,run!,<sos> courez ! <eos>
8,run!,<sos> prenez vos jambes à vos cous ! <eos>
9,run!,<sos> file ! <eos>


In [6]:
# 영어 토큰화
eng_tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3] # 0,1,2모두 go 기 때문에 같을 수 밖에 없으니 당황하지 말 것

[[22], [22], [22]]

In [7]:
# 프랑스어 토큰화
fra_tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 59, 2], [1, 351, 2], [1, 22, 499, 2]]

In [8]:
# 디코더 토큰 중 디코더 입력 데이터에 종료 토큰과 디코더 target 데이터에 시작 토큰 제거
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [9]:
# 패딩 길이 확인을 위해 영어와 프랑스어 시퀸스의 최대 길이를 구한다.(단어 단위)
max_eng_seq_len=max([len(k_len) for k_len in input_text])
max_fra_seq_len=max([len(f_len) for f_len in target_text])
print("영어 최대 길이:", max_eng_seq_len)
print("프랑스어 최대 길이:", max_fra_seq_len)

영어 최대 길이: 6
프랑스어 최대 길이: 14


In [10]:
encoder_input= pad_sequences(encoder_input, maxlen=max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 6)
프랑스어 입력데이터의 크기(shape) : (33000, 14)
프랑스어 출력데이터의 크기(shape) : (33000, 14)


In [11]:
print(encoder_input[0])

[22  0  0  0  0  0]


In [12]:
# 각 단어장 크기 확인
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4799
프랑스어 단어장의 크기 : 10009


In [13]:
# encoder_input = to_categorical(encoder_input)
# decoder_input = to_categorical(decoder_input)
# decoder_target = to_categorical(decoder_target)
# print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
# print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
# print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

In [14]:
# 노드와 마찬가지로 3000건만 검증데이터로 준다.
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

np.shape(encoder_input_train)

영어 학습데이터의 크기(shape) : (33000, 6)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 14)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 14)


(30000, 6)

In [15]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

### 모델 형성 및 훈련

In [16]:
encoder_inputs = Input(shape=(None,))
# 단어장의 크기 및 임베딩 벡터 차원입력
enc_emb =  Embedding(4799, 8)(encoder_inputs)
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

In [17]:
# 디코더 입력 텐서 생성.(교사 강요)
decoder_inputs = Input(shape=(None,))
# 단어장의 크기 및 임베딩 벡터 차원입력
fra_emb= Embedding(10009, 8)(decoder_inputs)
decoder_lstm = LSTM(units=256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(fra_emb)

In [18]:
# 결과 중 가장 확률이 높은 하나의 단어만 선택
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [19]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 8)      80072       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  271360      embedding_1[0][0]                
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [21]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
235/235 [==============================] - 7s 19ms/step - loss: 2.5212 - val_loss: 2.2431
Epoch 2/50
235/235 [==============================] - 4s 17ms/step - loss: 1.7302 - val_loss: 1.9863
Epoch 3/50
235/235 [==============================] - 4s 17ms/step - loss: 1.6052 - val_loss: 1.9105
Epoch 4/50
235/235 [==============================] - 4s 17ms/step - loss: 1.5489 - val_loss: 1.8731
Epoch 5/50
235/235 [==============================] - 4s 17ms/step - loss: 1.5105 - val_loss: 1.8504
Epoch 6/50
235/235 [==============================] - 4s 17ms/step - loss: 1.4796 - val_loss: 1.8468
Epoch 7/50
235/235 [==============================] - 4s 17ms/step - loss: 1.4503 - val_loss: 1.8122
Epoch 8/50
235/235 [==============================] - 4s 17ms/step - loss: 1.4243 - val_loss: 1.7960
Epoch 9/50
235/235 [==============================] - 4s 17ms/step - loss: 1.4025 - val_loss: 1.7762
Epoch 10/50
235/235 [==============================] - 4s 17ms/step - loss: 1.3839 - val_lo

### 모델 테스트하기

In [23]:
encoder_model = Model(inputs = encoder_inputs, outputs = [state_h, state_c])
encoder_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 8)           38392     
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 271360    
Total params: 309,752
Trainable params: 309,752
Non-trainable params: 0
_________________________________________________________________


In [24]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# decoder_lstm에 fra_emb 적용.
decoder_outputs, state_h, state_c = decoder_lstm(fra_emb, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [25]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 8)      80072       input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
____________________________________________________________________________________________

In [26]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [37]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음(리스트)
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성(3차원 matrix)
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['<sos>']] = 1.
    
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [38]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

ValueError: in user code:

    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:1569 run_step  **
        outputs = model.predict_step(data)
    /opt/conda/lib/python3.9/si